In [9]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info.operators import Operator
import string
import numpy as np
from scipy.linalg import expm

In [53]:
######################
# transfer matrix right environment
######################
all_params = [2*np.pi * np.random.random(15) for _ in range(3)]
U1 = generalU(all_params[0], 'U').to_instruction()
U2 = generalU(all_params[1], 'U').to_instruction()
U3 = generalU(all_params[2], 'U').to_instruction()

qr = QuantumRegister(8) # quantum - register
circ = QuantumCircuit(qr, name = 'circuit')

circ.append(U3, [qr[4], qr[5]])

circ.append(U2, [qr[3], qr[4]])

circ.append(U1, [qr[2], qr[3]])

print(circ.draw(fold = -1))
print(circ.decompose().depth())

                             
q171974_0: ──────────────────
                             
q171974_1: ──────────────────
                       ┌────┐
q171974_2: ────────────┤0   ├
                 ┌────┐│  U │
q171974_3: ──────┤0   ├┤1   ├
           ┌────┐│  U │└────┘
q171974_4: ┤0   ├┤1   ├──────
           │  U │└────┘      
q171974_5: ┤1   ├────────────
           └────┘            
q171974_6: ──────────────────
                             
q171974_7: ──────────────────
                             
33


In [51]:
all_params = [2*np.pi * np.random.random(15) for _ in range(6)]
U1 = generalU(all_params[0], 'U1').to_instruction()
U2 = generalU(all_params[1], 'U2').to_instruction()
U3 = generalU(all_params[2], 'U3').to_instruction()
U1_ = generalU(all_params[0], 'U1\'').to_instruction()
U2_ = generalU(all_params[1], 'U2\'').to_instruction()
U3_ = generalU(all_params[2], 'U3\'').to_instruction()

Ut = time_evo_op(0.1)

qr = QuantumRegister(8) # quantum - register
circ = QuantumCircuit(qr, name = 'circuit')

circ.append(U3, [qr[0], qr[1]])
circ.append(U3, [qr[2], qr[3]])
circ.append(U3, [qr[4], qr[5]])
circ.append(U3, [qr[6], qr[7]])

circ.append(U2, [qr[1], qr[2]])
circ.append(U2, [qr[3], qr[4]])
circ.append(U2, [qr[5], qr[6]])


circ.append(U1, [qr[2], qr[3]])
circ.append(U1, [qr[4], qr[5]])

circ.barrier()
circ.unitary(Ut, [2,5])
circ.barrier()
circ.append(U1_.inverse(), [qr[2], qr[3]])
circ.append(U1_.inverse(), [qr[4], qr[5]])


circ.append(U2_.inverse(), [qr[1], qr[2]])
circ.append(U2_.inverse(), [qr[3], qr[4]])
circ.append(U2_.inverse(), [qr[5], qr[6]])


circ.append(U3_.inverse(), [qr[0], qr[1]])
circ.append(U3_.inverse(), [qr[2], qr[3]])
circ.append(U3_.inverse(), [qr[4], qr[5]])
circ.append(U3_.inverse(), [qr[6], qr[7]])


print(circ.draw(fold = -1))
print(circ.decompose().depth())

           ┌─────┐               ░              ░                       ┌─────────┐
q171965_0: ┤0    ├───────────────░──────────────░───────────────────────┤0        ├
           │  U3 │┌─────┐        ░              ░            ┌─────────┐│  U3'_dg │
q171965_1: ┤1    ├┤0    ├────────░──────────────░────────────┤0        ├┤1        ├
           ├─────┤│  U2 │┌─────┐ ░ ┌──────────┐ ░ ┌─────────┐│  U2'_dg │├─────────┤
q171965_2: ┤0    ├┤1    ├┤0    ├─░─┤0         ├─░─┤0        ├┤1        ├┤0        ├
           │  U3 │├─────┤│  U1 │ ░ │          │ ░ │  U1'_dg │├─────────┤│  U3'_dg │
q171965_3: ┤1    ├┤0    ├┤1    ├─░─┤          ├─░─┤1        ├┤0        ├┤1        ├
           ├─────┤│  U2 │├─────┤ ░ │  Unitary │ ░ ├─────────┤│  U2'_dg │├─────────┤
q171965_4: ┤0    ├┤1    ├┤0    ├─░─┤          ├─░─┤0        ├┤1        ├┤0        ├
           │  U3 │├─────┤│  U1 │ ░ │          │ ░ │  U1'_dg │├─────────┤│  U3'_dg │
q171965_5: ┤1    ├┤0    ├┤1    ├─░─┤1         ├─░─┤1        ├┤0        ├┤1  

In [48]:
from scipy.optimize import minimize, approx_fprime
from qiskit import Aer, execute
backend = Aer.get_backend('statevector_simulator')
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info.operators import Operator
import string
import numpy as np
from scipy.linalg import expm
from functools import partial

################################
# Make a generalised 2 qubit gate
################################

def generalU(param_vals, name):
    assert len(param_vals) == 15

    sub_qr = QuantumRegister(2) # quantum - register
    sub_circ = QuantumCircuit(sub_qr, name = name)

    parameters = [Parameter(symbol) for symbol in list(map(chr, range(97, 97+15)))]

    sub_circ.rz(parameters[0], 0)
    sub_circ.ry(parameters[1], 0)
    sub_circ.rz(parameters[2], 0)

    sub_circ.rz(parameters[3], 1)
    sub_circ.ry(parameters[4], 1)
    sub_circ.rz(parameters[5], 1)

    sub_circ.cx(1,0)

    sub_circ.rz(parameters[6], 0)
    sub_circ.ry(parameters[7], 1)

    sub_circ.cx(0,1)

    sub_circ.ry(parameters[8],1 )

    sub_circ.cx(1,0)

    sub_circ.rz(parameters[9], 0)
    sub_circ.ry(parameters[10], 0)
    sub_circ.rz(parameters[11], 0)

    sub_circ.rz(parameters[12], 1)
    sub_circ.ry(parameters[13], 1)
    sub_circ.rz(parameters[14], 1)

    circ1 = sub_circ.bind_parameters({p:v for p,v in zip(parameters, param_vals)} )
    return circ1

def zerosU(param_vals, name):
    assert len(param_vals) == 3
    """
    Circuit to define unitaries that are connected to the all zeros state. Because of this 
    it can be parameterised with far fewer parameters:
    
    0   0
    |   |
    H   |
    @---X
    |   |
    |  |U|
    |   |
    
    Where U is a general 1 qubit unitary which can be specified with just 3 parameters
    """
    sub_qr = QuantumRegister(2) # quantum - register
    sub_circ = QuantumCircuit(sub_qr, name = name)

    parameters = [Parameter(symbol) for symbol in list(map(chr, range(97, 97+3)))]
    
    sub_circ.h(1)
    sub_circ.cx(1,0)
    sub_circ.rz(parameters[0],0)
    sub_circ.ry(parameters[1],0)
    sub_circ.rz(parameters[2],0)
    
    circ1 = sub_circ.bind_parameters({p:v for p,v in zip(parameters, param_vals)} )

    return circ1


def time_evo_op(dt):

    Z = np.array([
        [1,0],
        [0,-1]
    ])

    X = np.array([
        [0,1],
        [1,0]
    ])

    I = np.array([
        [1,0],
        [0,1]
    ])

    ZZ = np.kron(Z,Z)
    XI = np.kron(X,I)
    IX = np.kron(I,X)
    t = dt
    expH = expm( -1j*(ZZ + 0.5*(IX + XI) ) * t  )

    return Operator(expH)

def cost_func_circ( var_params, fixed_params, time_evo):

    U1 = generalU(fixed_params, 'U1').to_instruction()
    U2 = generalU(fixed_params, 'U2').to_instruction()
    U3 = generalU(fixed_params, 'U3').to_instruction()
    U1_ = generalU(var_params, 'U1\'').to_instruction()
    U2_ = generalU(var_params, 'U2\'').to_instruction()
    U3_ = generalU(var_params, 'U3\'').to_instruction()

    qr = QuantumRegister(8) # quantum - register
    circ = QuantumCircuit(qr, name = 'circuit')

    circ.append(U3, [qr[0], qr[1]])
    circ.append(U3, [qr[2], qr[3]])
    circ.append(U3, [qr[4], qr[5]])
    circ.append(U3, [qr[6], qr[7]])

    circ.append(U2, [qr[1], qr[2]])
    circ.append(U2, [qr[3], qr[4]])
    circ.append(U2, [qr[5], qr[6]])

    circ.append(U1, [qr[2], qr[3]])
    circ.append(U1, [qr[4], qr[5]])

    circ.barrier()

    circ.unitary(time_evo, [3,4], label = 'e^iHt')

    circ.barrier()
    circ.append(U1_.inverse(), [qr[2], qr[3]])
    circ.append(U1_.inverse(), [qr[4], qr[5]])

    circ.append(U2_.inverse(), [qr[1], qr[2]])
    circ.append(U2_.inverse(), [qr[3], qr[4]])
    circ.append(U2_.inverse(), [qr[5], qr[6]])

    circ.append(U3_.inverse(), [qr[0], qr[1]])
    circ.append(U3_.inverse(), [qr[2], qr[3]])
    circ.append(U3_.inverse(), [qr[4], qr[5]])
    circ.append(U3_.inverse(), [qr[6], qr[7]])
    
    return circ

def cost_function(var_params, other_params):
    """
    var_params are variationally adjusted to find maximum overlap
    
    other params has the following form:
    [fixed_params, time_evo_operator, backend]
    """
    
    circ = cost_func_circ(var_params, other_params[0], other_params[1])
    
    job = execute(circ, other_params[2])
    result = job.result()
    psi = result.get_statevector(circ)
    return -np.abs(psi[0])


timeEvo = time_evo_op(1)
fixed_params = np.random.randn(15)
backend = Aer.get_backend('statevector_simulator')
def callback(xk): 
    print(cost_function(xk, [fixed_params, timeEvo, backend]))
    return True

cf = partial(cost_function, other_params = [fixed_params, timeEvo, backend])

def gradient(xk, *args):
    return approx_fprime(xk, f = cf, epsilon = 0.01)



    
r_params = minimize(cost_function, x0 = fixed_params, args = ([fixed_params, timeEvo, backend]), method = 'BFGS', options = {'disp':True}, callback = callback, jac = gradient, tol=1e-4)

-0.3670785021661418
-0.3954389763205755
-0.41710760623016124
-0.42654952590366707
-0.43171811091218426
-0.44056682655797685


KeyboardInterrupt: 

Notes on the previous section:

1) It appears like there is a lot of redundency in the parametrisation of the MPS in this way. The Nealder mead algorithm seems to make a lot of alterations that do not affect the cost function. As such there must be a better choice with fewer parameters to specify the algorithms. I have already foudn such a compact representation for U3